In [1]:
import os
import geopandas as gpd
import pandas as pd
import folium
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

In [2]:
class Mapper:
    def __init__(self, shp=None, data=None, maps={}, years=None, periods=None, variables=None):
        self.shp = shp
        self.data = data
        self.maps = maps
        self.years = years
        self.periods = periods
        self.variables = variables

    def getGDF(self):
        print(os.getcwd())
        shp = gpd.read_file("C:/Users/cschr/Documents/GitHub/DS5110_MAP21/Data/Virginia2020/Virginia.shp")
        data = pd.read_csv("C:/Users/cschr/Documents/GitHub/DS5110_MAP21/Data/Final_Data/trainableData.csv") 
        gdf = gpd.GeoDataFrame(data.merge(shp[['Tmc', 'TmcType', 'RoadNumber', 'IsPrimary', 'geometry']], left_on=['TMC'], right_on=['Tmc'], how='left')).set_crs("EPSG:4326")
        return gdf

    def coords(self, geom):
        return [(point[1],point[0]) for point in geom.coords]

    def quickMap(SELF, gdf, col):
        if pd.api.types.is_numeric_dtype(gdf[col]):
            vmin = gdf[col].quantile(.1)
            vmax = gdf[col].quantile(.9)
            return gdf.explore(col, tiles='cartoDB positron', cmap='Reds', vmin=0, vmax=vmax)
        else:
            return gdf.explore(col, tiles='cartoDB positron', cmap='bwr')

    def makePeriodByYearMap(self, period, year):
        gdf = self.getGDF()
        gdf['points'] = gdf.apply(lambda row: self.coords(row.geometry), axis=1)
        periodGDF = gdf.loc[gdf.PERIOD == period]
        obsmap = self.quickMap(periodGDF.loc[periodGDF.YEAR==year].drop('points', axis=1), 'obs rel_unrel')
        return obsmap
    
    ### ADDITIONS ###
    
    def createMapViews(self, gdf, _variables):
        display(Markdown(f"### Creating Map Views"))
        self.years = gdf.YEAR.unique()
        print(f"Possible Years: {self.years}")
        self.periods = gdf.PERIOD.unique()
        print(f"Possible Periods: {self.periods}")
        self.variables = _variables
        print(f"Possible Variables: {self.variables}")
        print(f"Creating Views:")
        for year in self.years:
            self.maps[year] = {}
            for period in self.periods:
                self.maps[year][period] = {}
                for variable in self.variables:
                    try:
                        temp_df = gdf.loc[(gdf.YEAR == year) & (gdf.PERIOD == period)][['TMC',variable,'geometry']]
                        self.maps[year][period][variable] = mapper.quickMap(temp_df, variable)
                        print(f"- Created: {year}, {period}, {variable}")
                    except:
                        print(f"- Failed: {year}, {period}, {variable}")
                        
    def addMapViews(self, gdf, variables):
        for variable in variables:
            self.variables.append(variable)
        display(Markdown(f"### Adding Map Views"))
        print(f"Additional Variables: {variables}")
        print(f"Creating Views:")
        for year in self.years:
            for period in self.periods:
                for variable in variables:
                    try:
                        temp_df = gdf.loc[(gdf.YEAR == year) & (gdf.PERIOD == period)][['TMC',variable,'geometry']]
                        self.maps[year][period][variable] = mapper.quickMap(temp_df, variable)
                        print(f"- Created: {year}, {period}, {variable}")
                    except:
                        print(f"- Failed: {year}, {period}, {variable}")
                        
    def viewMaps(self):
        @widgets.interact(year=self.years, period=self.periods, variable=self.variables)
        def changeMap(year, period, variable):
            return mapper.maps[year][period][variable]

In [3]:
# create mapper object
mapper = Mapper()

# pull data
gdf = mapper.getGDF()

# create the map views for selected variables
mapper.createMapViews(gdf, ['obs rel_unrel', 'Crashes', 'AVG_HOURLY_VAL', 'PCT-PRECIP-MINS'])

/sfs/qumulo/qhome/dbn5eu/ds5110/DS5110_MAP21/Code


DriverError: C:/Users/cschr/Documents/GitHub/DS5110_MAP21/Data/Virginia2020/Virginia.shp: No such file or directory

In [4]:
%%time
mapper.viewMaps()

interactive(children=(Dropdown(description='year', options=(2017, 2018, 2019, 2020), value=2017), Dropdown(des…

Wall time: 1 s


In [5]:
%%time
mapper.addMapViews(gdf, ['ALL_WEATHER', 'TRUCK_PCT'])

### Adding Map Views

Additional Variables: ['ALL_WEATHER', 'TRUCK_PCT']
Creating Views:
- Created: 2017, AMP, ALL_WEATHER
- Created: 2017, AMP, TRUCK_PCT
- Created: 2017, MIDD, ALL_WEATHER
- Created: 2017, MIDD, TRUCK_PCT
- Created: 2017, PMP, ALL_WEATHER
- Created: 2017, PMP, TRUCK_PCT
- Created: 2017, WE, ALL_WEATHER
- Created: 2017, WE, TRUCK_PCT
- Created: 2018, AMP, ALL_WEATHER
- Created: 2018, AMP, TRUCK_PCT
- Created: 2018, MIDD, ALL_WEATHER
- Created: 2018, MIDD, TRUCK_PCT
- Created: 2018, PMP, ALL_WEATHER
- Created: 2018, PMP, TRUCK_PCT
- Created: 2018, WE, ALL_WEATHER
- Created: 2018, WE, TRUCK_PCT
- Created: 2019, AMP, ALL_WEATHER
- Created: 2019, AMP, TRUCK_PCT
- Created: 2019, MIDD, ALL_WEATHER
- Created: 2019, MIDD, TRUCK_PCT
- Created: 2019, PMP, ALL_WEATHER
- Created: 2019, PMP, TRUCK_PCT
- Created: 2019, WE, ALL_WEATHER
- Created: 2019, WE, TRUCK_PCT
- Created: 2020, AMP, ALL_WEATHER
- Created: 2020, AMP, TRUCK_PCT
- Created: 2020, MIDD, ALL_WEATHER
- Created: 2020, MIDD, TRUCK_PCT
- Creat

In [6]:
%%time
mapper.viewMaps()

interactive(children=(Dropdown(description='year', options=(2017, 2018, 2019, 2020), value=2017), Dropdown(des…

Wall time: 874 ms
